In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from torch.optim.optimizer import Optimizer

In [2]:
class SGD(Optimizer):
    """SGD
    """

    def __init__(
        self,
        params,
        lr = 1e-3
    ):
        
        defaults = dict(
            lr=lr
        )
        super(SGD, self).__init__(params, defaults)

    def step(self):
        """Performs a single optimization step.
        """
        for group in self.param_groups:
            lr = group["lr"]
            
            for p in group['params']:
                # the grad attribute is set by the backwards pass... here we just need to use it's value
                p -= lr * p.grad


In [89]:
import torch

class Adam(Optimizer):
    """Adam optimizer
    """

    def __init__(
        self,
        params,
        lr= 1e-3,
        betas = (0.9, 0.999),
        eps = 1e-8
    ):

        defaults = dict(
            lr=lr,
            betas=betas,
            eps=eps,)
        super(Adam, self).__init__(params, defaults)
        # initialize hyper parameters for every parameter
        self.param_groups = []
        for param in params:
            print(param)
            d = {}
            d["lr"] = torch.empty(size=(param.shape)).fill(lr)
            d["betas"] = (torch.empty(size=(param.shape)).fill(betas[0]), torch.empty(size=(param.shape)).fill(betas[0]))
            d["eps"] = torch.empty(size=(param.shape)).fill(eps)

    def __setstate__(self, state):
        super(Adam, self).__setstate__(state)

    def step(self):
        r"""Performs a single optimization step.
        """

        for group in self.param_groups:
            lr = group['lr']
            beta1, beta2 = group['betas']
            eps = group['eps']

            for p in group['params']:
                #TODO store the optimizer state (step, exp_avg, exp_avg_sq) in self.state[p] (create before first step)
                
                #TODO update optimizer state
                
                #TODO calculate debiased exp_av and exp_avg_sq
                
                #TODO perform the actual update